In [129]:
import os
import sys
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# path to the chromedriver executable
chromedriver = "~/Downloads/chromedriver"
chromedriver = os.path.expanduser(chromedriver)
sys.path.append(chromedriver)

#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--proxy-server=10.252.23.135')
#driver = webdriver.Chrome(chromedriver, chrome_options=chrome_options)

driver = webdriver.Chrome(chromedriver)
driver.get("https://shop.lego.com/en-LU/Themes")
time.sleep(5)

In [147]:
def find_attribute(class_name):
    """
    Try to find an element, but return null if element is not present
    """
    try:
        var_name = driver.find_element_by_class_name(class_name).text
    except:
        var_name = np.nan
        
    return var_name

def find_attributes(class_name, index):
    """
    Try to find an element, but return null if element is not present
    """
    try:
        var_name = [i.text for i in driver.find_elements_by_class_name(class_name)][index]
    except:
        var_name = np.nan
        
    return var_name

In [148]:
all_prods = []

#find all the themes of sets
theme_buttons = driver.find_elements_by_class_name("cat-list__see-products")
for g in range(0,len(theme_buttons)):
    
    #find all the themes of sets
    theme_buttons = driver.find_elements_by_class_name("cat-list__see-products")
    
    #get theme name
    themes = driver.find_elements_by_class_name("cat-list__title")
    theme_name = themes[g].text

    #click into the first theme
    button_loc = theme_buttons[g].location['y'] - 500
    driver.execute_script("window.scrollTo(0," + str(button_loc) + " )")
    theme_buttons[g].click()
    time.sleep(5)
    
    #expand all items
    try:
        driver.execute_script("window.scrollTo(0, 2500)")
        driver.find_element_by_class_name("pagination__view-all").click()
        time.sleep(5)
    except:
        pass
    
    #get all product_links
    prod_links = [i.get_attribute("href") for i in driver.find_elements_by_class_name("product-leaf__link-title")]
    
    #go back to the themes page
    if len(prod_links) > 18:
        
        #go back to the rest of the themes
        driver.execute_script("window.history.go(-2)")
    else:
        
        #go back to the rest of the themes
        driver.execute_script("window.history.go(-1)")
    
    time.sleep(5)
    all_prods.append(prod_links)
all_prods = sum(all_prods, [])

In [149]:
#intialize variables to scrape
theme = []
set_name = []
list_price = []
prod_id = []
ages = []
piece_count = []
prod_desc = []
star_rating = []
play_star_rating = []
val_star_rating = []
num_reviews = []
review_difficulty = []
prod_long_desc = []

for prod in all_prods:
    
    #go to page
    driver.get(prod)
    time.sleep(1)
    
    #scrape data
    theme.append(find_attributes('markup', 1))
    set_name.append(find_attribute('overview__name'))
    list_price.append(find_attribute('product-price__list-price'))
    prod_id.append(find_attribute('product-details__product-code'))
    ages.append(find_attribute('product-details__ages'))
    piece_count.append(find_attribute('product-details__piece-count'))
    prod_desc.append(find_attribute('product-details__description'))
    star_rating.append(find_attributes('BVRRRatingNumber', 0))
    play_star_rating.append(find_attributes('BVRRRatingNumber', 1))
    val_star_rating.append(find_attributes('BVRRRatingNumber', 2))
    num_reviews.append(find_attribute('BVRRNumber'))
    review_difficulty.append(find_attribute('BVRRSliderTextDisplayValue'))
    prod_long_desc.append(find_attribute('product-features__description'))

In [150]:
pd.DataFrame({
    'theme_name':theme,
    'set_name':set_name,
    'list_price':list_price,
    'prod_id':prod_id,
    'ages':ages,
    'piece_count':piece_count,
    'prod_desc':prod_desc,
    'prod_long_desc':prod_long_desc,
    'star_rating':star_rating,
    'play_star_rating':play_star_rating,
    'val_star_rating':val_star_rating,
    'num_reviews':num_reviews,
    'review_difficulty':review_difficulty
             }).to_csv('..//data//AT_data.csv', index = False)

In [128]:
#close connection
driver.close()